In [ ]:
# Prob Stmt : To find insights from the data to generate value for the business.
# Client wants to collect and analyse some diff datasets to find some pattern and make music based on that.
# Client wants to collect top trending global songs playlist data on weekly basis. Hence, asked to make ETL pipeline.
# Spotify is a music streaming app.

In [ ]:
                   # Architecture
# Spotify API - (Music Artists,Albums & Tracks) , Get client id & secret key id from dev account to extract data
# Note: With python Application package will connect with Spotify API to extract data
# AWS Lambda - Write and Deploy code on it (Data extraction and data transformation code)
# AWS CloudWatch - Scheduler to run code on weekly basis or specific time or set alarms & monitor logs
# AWS S3 - Store raw data in it 
# Trigger - using it atomatically trigger transformation fn/code to transform data and put back in S3
# Glue Crawler - Once data in S3 it will infer schema to create AWS Glue Data Catalog(metadata repository)
# AWS Athena - Once data in Data Catalog use athena to run SQL queries on top of it or in S3 buckets

In [2]:
#python library to for Spotify Web API
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 5.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [145]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [102]:
# Provide details for authentication to extract data from spotify
client_credentials_mgr = SpotifyClientCredentials(client_id='74e69899708441629f3782138d5bb9e2',
                                                      client_secret='9fb83d5ed5fa4f359554e19b3805ff3f')

In [103]:
# Create an object to provides authorization to access data to extract
sp = spotipy.Spotify(client_credentials_manager = client_credentials_mgr)

In [104]:
playlist_link = 'https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF'

In [105]:
playlist_URI = playlist_link.split('/')[-1]

In [116]:
# To extract all the info related to tracks
data = sp.playlist_tracks(playlist_URI)

In [121]:
data

{'href': 'https://api.spotify.com/v1/playlists/37i9dQZEVXbNG2KDcFcKOF/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2023-04-28T12:56:04Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_group': 'single',
     'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0XeEobZplHxzM9QzFQWLiR'},
       'href': 'https://api.spotify.com/v1/artists/0XeEobZplHxzM9QzFQWLiR',
       'id': '0XeEobZplHxzM9QzFQWLiR',
       'name': 'Eslabon Armado',
       'type': 'artist',
       'uri': 'spotify:artist:0XeEobZplHxzM9QzFQWLiR'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/12GqGscKJx3aE4t07u7eVZ'},
       'href': 'https://api.spotify.com/v1/artists/12GqGscKJx3aE4t07u7eVZ',
       '

In [112]:
# Extracting relevant data one by one
data['items'][0]['track']['album']['id']

'3Mm1P5CEEAiuJqBclr2EyU'

In [58]:
data['items'][0]['track']['album']['name']

'Ella Baila Sola'

In [68]:
data['items'][0]['track']['album']['release_date']

'2023-03-16'

In [75]:
data['items'][0]['track']['album']['total_tracks']

1

In [78]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/3Mm1P5CEEAiuJqBclr2EyU'

In [ ]:
data['items']

In [117]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                    'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)
album_list

In [135]:
artist_list = []
for row in data['items']:
    for key,value in row.items():
        if key == 'track':
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'],'artist_name':artist['name'],'externam_url':artist['external_urls']['spotify']}
                artist_list.append(artist_dict)
artist_list

[{'artist_id': '0XeEobZplHxzM9QzFQWLiR',
  'artist_name': 'Eslabon Armado',
  'externam_url': 'https://open.spotify.com/artist/0XeEobZplHxzM9QzFQWLiR'},
 {'artist_id': '12GqGscKJx3aE4t07u7eVZ',
  'artist_name': 'Peso Pluma',
  'externam_url': 'https://open.spotify.com/artist/12GqGscKJx3aE4t07u7eVZ'},
 {'artist_id': '6XkjpgcEsYab502Vr1bBeW',
  'artist_name': 'Grupo Frontera',
  'externam_url': 'https://open.spotify.com/artist/6XkjpgcEsYab502Vr1bBeW'},
 {'artist_id': '4q3ewBCX7sLwd24euuV69X',
  'artist_name': 'Bad Bunny',
  'externam_url': 'https://open.spotify.com/artist/4q3ewBCX7sLwd24euuV69X'},
 {'artist_id': '1NNRWkhwmcXRimFYSBpB1y',
  'artist_name': 'Yng Lvcas',
  'externam_url': 'https://open.spotify.com/artist/1NNRWkhwmcXRimFYSBpB1y'},
 {'artist_id': '12GqGscKJx3aE4t07u7eVZ',
  'artist_name': 'Peso Pluma',
  'externam_url': 'https://open.spotify.com/artist/12GqGscKJx3aE4t07u7eVZ'},
 {'artist_id': '5YGY8feqx7naU7z4HrwZM6',
  'artist_name': 'Miley Cyrus',
  'externam_url': 'https://

In [144]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_dict = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                'popularity':song_popularity,'song_added':song_added,'album_id':album_id,'artist_id':artist_id}
    song_list.append(song_dict)
song_list

[{'song_id': '3dnP0JxCgygwQH9Gm7q7nb',
  'song_name': 'Ella Baila Sola',
  'duration_ms': 165671,
  'url': 'https://open.spotify.com/track/3dnP0JxCgygwQH9Gm7q7nb',
  'popularity': 100,
  'song_added': '2023-04-28T12:56:04Z',
  'album_id': '3Mm1P5CEEAiuJqBclr2EyU',
  'artist_id': '0XeEobZplHxzM9QzFQWLiR'},
 {'song_id': '6pD0ufEQq0xdHSsRbg9LBK',
  'song_name': 'un x100to',
  'duration_ms': 194563,
  'url': 'https://open.spotify.com/track/6pD0ufEQq0xdHSsRbg9LBK',
  'popularity': 98,
  'song_added': '2023-04-28T12:56:04Z',
  'album_id': '7aGzSSUD8S6IhPCsZSiuMT',
  'artist_id': '6XkjpgcEsYab502Vr1bBeW'},
 {'song_id': '2UW7JaomAMuX9pZrjVpHAU',
  'song_name': 'La Bebe - Remix',
  'duration_ms': 234352,
  'url': 'https://open.spotify.com/track/2UW7JaomAMuX9pZrjVpHAU',
  'popularity': 98,
  'song_added': '2023-04-28T12:56:04Z',
  'album_id': '6aBVGuOUEuX18rHxyDWbti',
  'artist_id': '1NNRWkhwmcXRimFYSBpB1y'},
 {'song_id': '4DHcnVTT87F0zZhRPYmZ3B',
  'song_name': 'Flowers',
  'duration_ms': 20060

In [151]:
# Converting above datasets into dfs
album_df = pd.DataFrame.from_dict(album_list)
artist_df = pd.DataFrame.from_dict(artist_list)
song_df = pd.DataFrame.from_dict(song_list)

In [162]:
album_df = album_df.drop_duplicates(subset='album_id')
artist_df = artist_df.drop_duplicates(subset='artist_id')
song_df = song_df.drop_duplicates(subset='song_id')

In [163]:
album_df.head()

,album_id,name,release_date,total_tracks,url
0,3Mm1P5CEEAiuJqBclr2EyU,Ella Baila Sola,2023-03-16,1,https://open.spotify.com/album/3Mm1P5CEEAiuJqB...
1,7aGzSSUD8S6IhPCsZSiuMT,un x100to,2023-04-17,1,https://open.spotify.com/album/7aGzSSUD8S6IhPC...
2,6aBVGuOUEuX18rHxyDWbti,La Bebe (Remix),2023-03-17,2,https://open.spotify.com/album/6aBVGuOUEuX18rH...
3,0HiZ8fNXwJOQcrf5iflrdz,Endless Summer Vacation,2023-03-10,13,https://open.spotify.com/album/0HiZ8fNXwJOQcrf...
4,6NcI39WPu4kY6Tul11nhSv,Daylight,2023-04-14,1,https://open.spotify.com/album/6NcI39WPu4kY6Tu...


In [164]:
artist_df.head()

,artist_id,artist_name,externam_url
0,0XeEobZplHxzM9QzFQWLiR,Eslabon Armado,https://open.spotify.com/artist/0XeEobZplHxzM9...
1,12GqGscKJx3aE4t07u7eVZ,Peso Pluma,https://open.spotify.com/artist/12GqGscKJx3aE4...
2,6XkjpgcEsYab502Vr1bBeW,Grupo Frontera,https://open.spotify.com/artist/6XkjpgcEsYab50...
3,4q3ewBCX7sLwd24euuV69X,Bad Bunny,https://open.spotify.com/artist/4q3ewBCX7sLwd2...
4,1NNRWkhwmcXRimFYSBpB1y,Yng Lvcas,https://open.spotify.com/artist/1NNRWkhwmcXRim...


In [165]:
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id,artist_id
0,3dnP0JxCgygwQH9Gm7q7nb,Ella Baila Sola,165671,https://open.spotify.com/track/3dnP0JxCgygwQH9...,100,2023-04-28T12:56:04Z,3Mm1P5CEEAiuJqBclr2EyU,0XeEobZplHxzM9QzFQWLiR
1,6pD0ufEQq0xdHSsRbg9LBK,un x100to,194563,https://open.spotify.com/track/6pD0ufEQq0xdHSs...,98,2023-04-28T12:56:04Z,7aGzSSUD8S6IhPCsZSiuMT,6XkjpgcEsYab502Vr1bBeW
2,2UW7JaomAMuX9pZrjVpHAU,La Bebe - Remix,234352,https://open.spotify.com/track/2UW7JaomAMuX9pZ...,98,2023-04-28T12:56:04Z,6aBVGuOUEuX18rHxyDWbti,1NNRWkhwmcXRimFYSBpB1y
3,4DHcnVTT87F0zZhRPYmZ3B,Flowers,200600,https://open.spotify.com/track/4DHcnVTT87F0zZh...,90,2023-04-28T12:56:04Z,0HiZ8fNXwJOQcrf5iflrdz,5YGY8feqx7naU7z4HrwZM6
4,1odExI7RdWc4BT515LTAwj,Daylight,212953,https://open.spotify.com/track/1odExI7RdWc4BT5...,94,2023-04-28T12:56:04Z,6NcI39WPu4kY6Tul11nhSv,33NVpKoXjItPwUJTMZIOiY


In [166]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      50 non-null     object
 1   name          50 non-null     object
 2   release_date  50 non-null     object
 3   total_tracks  50 non-null     int64 
 4   url           50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [170]:
# Applying transformation to convert into datetime datatype

album_df['release_date'] = pd.to_datetime(album_df['release_date'])

In [171]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   album_id      50 non-null     object        
 1   name          50 non-null     object        
 2   release_date  50 non-null     datetime64[ns]
 3   total_tracks  50 non-null     int64         
 4   url           50 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.1+ KB


In [172]:
song_df['song_added'] = pd.to_datetime(song_df['song_added'])


In [173]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   song_id      50 non-null     object             
 1   song_name    50 non-null     object             
 2   duration_ms  50 non-null     int64              
 3   url          50 non-null     object             
 4   popularity   50 non-null     int64              
 5   song_added   50 non-null     datetime64[ns, UTC]
 6   album_id     50 non-null     object             
 7   artist_id    50 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 3.2+ KB
